# Week 1 - Linear Regression 1 | Polynomial & Interaction terms, Multi-collinearity, VIF

#### <font color='plum'> RESPONSES IN THIS COLOR

For Week 1, include concepts such as:
- linear regression with polynomial terms
- interaction terms
- multicollinearity
- variance inflation factor and regression
- categorical and continuous features.

# Imports

In [35]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt 
import kagglehub 
import time
import statsmodels.api as sm

from sklearn.linear_model import LinearRegression

from sklearn.model_selection import (
    train_test_split, 
)

from sklearn.metrics import (
    r2_score, 
    mean_squared_error, 
    root_mean_squared_error,
    accuracy_score, 

)
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.datasets import make_regression
from datetime         import datetime, timedelta
from tqdm             import tqdm
from typing import Any, Dict, List, Union, Tuple
from sklearn.base import BaseEstimator
from collections import Counter
from scipy.stats import randint
# from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from statsmodels.stats.outliers_influence import variance_inflation_factor

%matplotlib inline

from kagglehub              import KaggleDatasetAdapter


In [36]:

sns.set_theme(font_scale=0.8) 
# plt.rcParams['figure.figsize'] = (8, 4)
plt.rcParams['axes.titlesize']  = 10
plt.rcParams['axes.labelsize']  = 8
plt.rcParams['lines.linewidth'] = 0.5
# plt.rcParams['lines.markersize'] = 3
plt.rcParams['axes.edgecolor']  = 'gray'
plt.rcParams['xtick.color']     = 'gray'
plt.rcParams['ytick.color'] = 'gray'
plt.rcParams['xtick.color'] = 'gray'
plt.rcParams['ytick.color'] = 'gray'
plt.rcParams['ytick.labelsize'] = 8
plt.rcParams['xtick.labelsize'] = 8

# Utility Functions

In [37]:
# globals
random_state    = 42
test_size       = 0.2
n_jobs          = -1
font_size       = 8

In [38]:
def get_linreg_stats(model_name, y_test, y_pred):
    """
    Computes regression statistics for a linear regression model.

    Parameters:
        y_test (array-like): True target values.
        y_pred (array-like): Predicted target values.

    Returns:
        dict: A dictionary containing the computed regression statistics.
    """
    mse = mean_squared_error(y_test, y_pred)
    rmse = root_mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    return {
        'Model': model_name,
        "MSE": mse,
        "RMSE": rmse,
        "R^2": r2
    }


# 1. Dataset:  Parkinsons Telemonitoring

Tsanas, A. & Little, M. (2009). Parkinsons Telemonitoring [Dataset]. UCI Machine Learning Repository. https://doi.org/10.24432/C5ZS3N.

https://archive.ics.uci.edu/dataset/189/parkinsons+telemonitoring

<font color='plum'>
This dataset is composed of a range of biomedical voice measurements from 42 people with early-stage Parkinson's disease recruited to a six-month trial of a telemonitoring device for remote symptom progression monitoring. The recordings were automatically captured in the patient's homes.<br>

- The rows of the CSV file contain an instance corresponding to one voice recording. There are around 200 recordings per patient, the subject number of the patient is identified in the first column.
- Columns in the table contain subject number, subject age, subject gender, time interval from baseline recruitment date, motor UPDRS, total UPDRS, and 16 biomedical voice measures. 
- Each row corresponds to one of 5875 voice recordings from these individuals. 
- **Goal: predict the motor and total UPDRS scores (`motor_UPDRS` and `total_UPDRS`) from the 16 voice measures**.

### Load & Clean

In [39]:
# Install the ucimlrepo package (needed in this notebook environment)
# %pip install ucimlrepo

from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
parkinsons_telemonitoring = fetch_ucirepo(id=189) 
  
# data (as pandas dataframes) 
X = parkinsons_telemonitoring.data.features 
y = parkinsons_telemonitoring.data.targets 
  
# metadata 
parkinsons_telemonitoring.metadata


{'uci_id': 189,
 'name': 'Parkinsons Telemonitoring',
 'repository_url': 'https://archive.ics.uci.edu/dataset/189/parkinsons+telemonitoring',
 'data_url': 'https://archive.ics.uci.edu/static/public/189/data.csv',
 'abstract': "Oxford Parkinson's Disease Telemonitoring Dataset",
 'area': 'Health and Medicine',
 'tasks': ['Regression'],
 'characteristics': ['Tabular'],
 'num_instances': 5875,
 'num_features': 19,
 'feature_types': ['Integer', 'Real'],
 'demographics': ['Age', 'Sex'],
 'target_col': ['motor_UPDRS', 'total_UPDRS'],
 'index_col': ['subject#'],
 'has_missing_values': 'no',
 'missing_values_symbol': None,
 'year_of_dataset_creation': 2009,
 'last_updated': 'Fri Nov 03 2023',
 'dataset_doi': '10.24432/C5ZS3N',
 'creators': ['Athanasios Tsanas', 'Max Little'],
 'intro_paper': {'ID': 229,
  'type': 'NATIVE',
  'title': "Accurate Telemonitoring of Parkinson's Disease Progression by Noninvasive Speech Tests",
  'authors': 'A. Tsanas, Max A. Little, P. McSharry, L. Ramig',
  'venue

In [40]:
  
# variable information 
parkinsons_telemonitoring.variables


,name,role,type,demographic,description,units,missing_values
0,subject#,ID,Integer,None,Integer that uniquely identifies each subject,None,no
1,age,Feature,Integer,Age,Subject age,None,no
2,test_time,Feature,Continuous,None,Time since recruitment into the trial. The int...,None,no
3,Jitter(%),Feature,Continuous,None,Several measures of variation in fundamental f...,None,no
4,Jitter(Abs),Feature,Continuous,None,Several measures of variation in fundamental f...,None,no
5,Jitter:RAP,Feature,Continuous,None,Several measures of variation in fundamental f...,None,no
6,Jitter:PPQ5,Feature,Continuous,None,Several measures of variation in fundamental f...,None,no
7,Jitter:DDP,Feature,Continuous,None,Several measures of variation in fundamental f...,None,no
8,Shimmer,Feature,Continuous,None,Several measures of variation in amplitude,None,no
9,Shimmer(dB),Feature,Continuous,None,Several measures of variation in amplitude,None,no


In [41]:
# concatenate X and first column of y
X = pd.concat([X, y['motor_UPDRS']], axis=1)
y = y.drop(columns=['motor_UPDRS'])


In [42]:
X

,age,test_time,Jitter(%),Jitter(Abs),Jitter:RAP,Jitter:PPQ5,Jitter:DDP,Shimmer,Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,Shimmer:APQ11,Shimmer:DDA,NHR,HNR,RPDE,DFA,PPE,sex,motor_UPDRS
0,72,5.6431,0.00662,0.000034,0.00401,0.00317,0.01204,0.02565,0.230,0.01438,0.01309,0.01662,0.04314,0.014290,21.640,0.41888,0.54842,0.16006,0,28.199
1,72,12.6660,0.00300,0.000017,0.00132,0.00150,0.00395,0.02024,0.179,0.00994,0.01072,0.01689,0.02982,0.011112,27.183,0.43493,0.56477,0.10810,0,28.447
2,72,19.6810,0.00481,0.000025,0.00205,0.00208,0.00616,0.01675,0.181,0.00734,0.00844,0.01458,0.02202,0.020220,23.047,0.46222,0.54405,0.21014,0,28.695
3,72,25.6470,0.00528,0.000027,0.00191,0.00264,0.00573,0.02309,0.327,0.01106,0.01265,0.01963,0.03317,0.027837,24.445,0.48730,0.57794,0.33277,0,28.905
4,72,33.6420,0.00335,0.000020,0.00093,0.00130,0.00278,0.01703,0.176,0.00679,0.00929,0.01819,0.02036,0.011625,26.126,0.47188,0.56122,0.19361,0,29.187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5870,61,142.7900,0.00406,0.000031,0.00167,0.00168,0.00500,0.01896,0.160,0.00973,0.01133,0.01549,0.02920,0.025137,22.369,0.64215,0.55314,0.21367,0,22.485
5871,61,149.8400,0.00297,0.000025,0.00119,0.00147,0.00358,0.02315,0.215,0.01052,0.01277,0.01904,0.03157,0.011927,22.886,0.52598,0.56518,0.12621,0,21.988
5872,61,156.8200,0.00349,0.000025,0.00152,0.00187,0.00456,0.02499,0.244,0.01371,0.01456,0.01877,0.04112,0.017701,25.065,0.47792,0.57888,0.14157,0,21.495
5873,61,163.7300,0.00281,0.000020,0.00128,0.00151,0.00383,0.01484,0.131,0.00693,0.00870,0.01307,0.02078,0.007984,24.422,0.56865,0.56327,0.14204,0,21.007


In [43]:
y

,total_UPDRS
0,34.398
1,34.894
2,35.389
3,35.810
4,36.375
...,...
5870,33.485
5871,32.988
5872,32.495
5873,32.007


## OLS

In [44]:

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=test_size, 
                                                    random_state=random_state
                                                    )

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

dict_ols = get_linreg_stats('Linear Regression', y_test, y_pred)

print(f"Mean Squared Error: {dict_ols['MSE']:.4f}")
print(f"Root Mean Squared Error: {dict_ols['RMSE']:.4f}")
print(f"R^2 Score: {dict_ols['R^2']:.4f}")

results_df = pd.DataFrame([dict_ols])
results_df


Mean Squared Error: 9.9762
Root Mean Squared Error: 3.1585
R^2 Score: 0.9100


,Model,MSE,RMSE,R^2
0,Linear Regression,9.976243,3.158519,0.909972


## Polynomial Terms

In [45]:
# Step 1: Standardize the features
scaler          = StandardScaler()
X_train_scaled  = scaler.fit_transform(X_train)
X_test_scaled   = scaler.transform(X_test)

# Step 2: Create polynomial features from the *standardized* data
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly    = poly.fit_transform(X_train_scaled)
X_test_poly     = poly.transform(X_test_scaled)

# Get the names of the new polynomial features
new_feature_names = poly.get_feature_names_out(input_features=X.columns)

# Create and fit the linear regression model
model_poly = LinearRegression()
model_poly.fit(X_train_poly, y_train)

# Make predictions on the test data
y_pred_poly = model_poly.predict(X_test_poly)

# Create a DataFrame of feature names and coefficients
coef_df = pd.DataFrame({
    'Feature':      new_feature_names,
    'Coefficient':  model_poly.coef_.flatten()
})

dict_poly   = get_linreg_stats('Polynomial Reg(deg = 2)', y_test, y_pred_poly)
# print(dict_poly)
new_row     = pd.DataFrame([dict_poly])

# Concatenate with the existing results_df
results_df  = pd.concat([results_df, new_row], ignore_index=True)

coef_df


,Feature,Coefficient
0,age,0.218700
1,test_time,0.218155
2,Jitter(%),-3.828265
3,Jitter(Abs),3.570335
4,Jitter:RAP,14.387841
...,...,...
225,PPE sex,-0.697980
226,PPE motor_UPDRS,0.197777
227,sex^2,0.077621
228,sex motor_UPDRS,-1.319062


<font color='plum'> `PolynomialFeatures(degree=2)` transformed all of the existing features into a comprehensive set of new features. This new set includes:

- The original features.

- Polynomial terms (e.g., age squared, test_time squared).

- All possible interaction terms between every pair of the original features (e.g., `age` * `sex`, `test_time`*`NHR`).

This is why the model's performance was so strong! It was able to capture both the non-linear relationships (through the polynomial terms) and the joint effects of different features (through the interaction terms) all at once. 

## Interaction

In [51]:
X_inter = X.copy()

X_inter['age_motor_interaction']    = X_inter['age'] * X_inter['motor_UPDRS']
features_inter_total                = ["age", "sex", "motor_UPDRS", "age_motor_interaction"]
X_inter_total                       = X_inter[features_inter_total]
print(X_inter_total.head())
X_train_inter_total, X_test_inter_total, y_train, y_test = train_test_split(X_inter_total, y, 
                                                                            test_size       = 0.2, 
                                                                            random_state    = 42
                                                                            )

model_inter_total   = LinearRegression()
model_inter_total.fit(X_train_inter_total, y_train)

y_pred_inter        = model_inter_total.predict(X_test_inter_total)

dict                = get_linreg_stats("LinReg w/Interaction Terms", y_test, y_pred_inter)


results_df          = pd.concat([results_df, pd.DataFrame([dict])], ignore_index=True)
results_df

   age  sex  motor_UPDRS  age_motor_interaction
0   72    0       28.199               2030.328
1   72    0       28.447               2048.184
2   72    0       28.695               2066.040
3   72    0       28.905               2081.160
4   72    0       29.187               2101.464


,Model,MSE,RMSE,R^2
0,Linear Regression,9.976243,3.158519,0.909972
1,Polynomial Reg(deg = 2),6.299803,2.509941,0.943149
2,LinReg w/Interaction Terms,10.310088,3.210933,0.906960
3,LinReg w/Combined Features,10.063111,3.172241,0.909188
4,LinReg w/Interaction Terms,10.310088,3.210933,0.906960


## Variance Inflation Factor analysis

In [47]:
# Add a constant (intercept) to the features, which is required for VIF calculation
X = sm.add_constant(X)

# Create a DataFrame to store the VIF results
vif_data            = pd.DataFrame()
vif_data["feature"] = X.columns
vif_data["VIF"]     = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]

# Print the results
print("Multicollinearity Analysis with Variance Inflation Factor (VIF)")
print("-" * 35)
print(vif_data)

Multicollinearity Analysis with Variance Inflation Factor (VIF)
-----------------------------------
          feature           VIF
0           const  6.539206e+02
1             age  1.147927e+00
2       test_time  1.017298e+00
3       Jitter(%)  8.907258e+01
4     Jitter(Abs)  7.951087e+00
5      Jitter:RAP  1.324277e+06
6     Jitter:PPQ5  3.103483e+01
7      Jitter:DDP  1.324518e+06
8         Shimmer  1.737054e+02
9     Shimmer(dB)  7.691367e+01
10   Shimmer:APQ3  2.398947e+07
11   Shimmer:APQ5  5.264356e+01
12  Shimmer:APQ11  1.532028e+01
13    Shimmer:DDA  2.398942e+07
14            NHR  8.593819e+00
15            HNR  5.480609e+00
16           RPDE  2.101023e+00
17            DFA  1.711742e+00
18            PPE  4.486241e+00
19            sex  1.353901e+00
20    motor_UPDRS  1.185992e+00


In [48]:
df = X.copy()
df = df.drop(columns=['const'])  # Drop the constant column added for VIF calculation

# Define a list of Jitter and Shimmer features to combine
jitter_features     = ['Jitter(%)', 'Jitter(Abs)', 'Jitter:RAP', 'Jitter:PPQ5', 'Jitter:DDP']
shimmer_features    = ['Shimmer', 'Shimmer(dB)', 'Shimmer:APQ3', 'Shimmer:APQ5', 'Shimmer:APQ11', 'Shimmer:DDA']

# Create new composite features by taking the mean of the highly correlated ones
df['Jitter_Composite']  = df[jitter_features].mean(axis=1)
df['Shimmer_Composite'] = df[shimmer_features].mean(axis=1)

# Drop the original highly correlated features
df_combined = df.drop(columns=jitter_features + shimmer_features)

# Define the target variable (y) and features (X) for the new model

X           = df_combined

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Perform Linear Regression on the new feature set
model           = LinearRegression()
model.fit(X_train, y_train)
y_pred         = model.predict(X_test)

dict_vif    = get_linreg_stats("LinReg w/Combined Features", y_test, y_pred)
results_df  = pd.concat([results_df, pd.DataFrame([dict_vif])], ignore_index=True)



In [49]:

# Perform a new VIF analysis on the combined feature set
X_vif               = sm.add_constant(X)
vif_data            = pd.DataFrame()
vif_data["feature"] = X_vif.columns
vif_data["VIF"]     = [variance_inflation_factor(X_vif.values, i) for i in range(len(X_vif.columns))]

print("\nMulticollinearity Analysis with Combined Features")
print("-" * 50)
print(vif_data)


Multicollinearity Analysis with Combined Features
--------------------------------------------------
              feature         VIF
0               const  613.574884
1                 age    1.138785
2           test_time    1.011585
3                 NHR    5.460644
4                 HNR    5.180710
5                RPDE    1.935764
6                 DFA    1.570026
7                 PPE    3.304111
8                 sex    1.141272
9         motor_UPDRS    1.155447
10   Jitter_Composite    4.287771
11  Shimmer_Composite    4.219242


## Summary

In [50]:
results_df

print(results_df)

                        Model        MSE      RMSE       R^2
0           Linear Regression   9.976243  3.158519  0.909972
1     Polynomial Reg(deg = 2)   6.299803  2.509941  0.943149
2  LinReg w/Interaction Terms  10.310088  3.210933  0.906960
3  LinReg w/Combined Features  10.063111  3.172241  0.909188


## NOTES

 ### <font color='gold'> Why do interaction terms require a large sample size? </font>

<font color='plum'>

Remember, interaction terms are always *added* to a feature set, and building a regression model from that augmented feature set requires a larger sample size b/c:

**1. More Complexity, More Parameters**
- Each interaction term added to the model introduces a new parameter to be estimated. 
- A model with `n` features and all possible two-way interaction terms will have `n` original terms and `n*(n-1)/2` interaction terms for a total of  `n + n(n−1)/2`. 
- Each new parameter requires data to be accurately estimated. A large number of parameters relative to the sample size can lead to an unstable model where the coefficients are highly sensitive to small changes in the data. If the sample size is too small, the model may overfit the data, leading to a complex but unreliable result that doesn't generalize well to new data.

**2. Sparseness of Data**
- Interaction terms are meant to capture nuanced relationships that only exist when two specific conditions are met simultaneously (e.g., the effect of a treatment is different for older vs. younger patients). 
- If the sample size is small, there may not be enough observations for every possible combination of these conditions. This makes the data for the interaction term sparse or non-existent, making it impossible to get a reliable estimate of its effect.

**3. Reduced Statistical Power**
- The more parameters in the model, the fewer degrees of freedom for residual error, which reduces the statistical power of the hypothesis tests, making it harder to determine if an interaction term's coefficient is statistically significant. A larger sample size helps to increase the degrees of freedom and, in turn, the power to detect real effects.

- More parameters → fewer residual degrees of freedom
- Fewer residual degrees of freedom → less precise error estimates
- Less precise error estimates → lower statistical power
- Lower power → harder to detect significance of coefficients (especially for interaction terms)

More complex models require more data to be properly trained and validated. Interaction terms dramatically increase the complexity of a model, demanding a larger sample size to ensure that the estimated relationships are both meaningful and statistically sound.

### <font color='gold'> When do polynomial terms tend to produce multicollinearity? </font>

<font color='plum'>

**Multicollinearity**: when independent variables in a regression model are correlated. 

Polynomial terms tend to produce multicollinearity when the original features are **not centered** around zero.

When you raise a variable to a power, the new polynomial term is almost perfectly correlated with the original variable, especially if the values of the original variable are far from zero.

For example, if you have a feature `age` and its values range from 30 to 80, the values for `age^2` will range from 900 to 6,400. As `age` increases, `age^2` also increases in a highly predictable way, leading to very high correlation between the two. This high correlation is the definition of multicollinearity.

**Why Centering Helps**

To reduce this effect, a common practice is to **standardize or center** your features first by subtracting the mean of the feature from each value.

Let's use our `age` example. If the average age is 55, centering the feature would make 55 become 0. Now, when you square the centered `age` feature, positive and negative values (for ages above and below the mean) can exist. This breaks the predictable, linear relationship, significantly reducing the correlation between `age` and `age^2`, and thus lowering the VIF score.


### <font color='gold'> Why would we use a VIF instead of correlation to detect multicollinearity? </font>
<font color = 'plum'>

We consider **VIF** alongside correlation b/c **VIF can detect multicollinearity among three or more variables, while correlation can only detect it between two**.

**The Limitation of Correlation** : A simple correlation coefficient measures the linear relationship between a single pair of variables. For example, you can calculate the correlation between `age` and `test_time`. If the correlation is high, you know those two are related.

However, multicollinearity often exists in more complex ways. A feature might have a low correlation with any *single* other feature, but it could be a perfect linear combination of *two or more* of them. Simple correlation would miss this entirely.

**The Power of VIF**:  **Variance Inflation Factor (VIF)** is a much more comprehensive diagnostic tool because it looks at the bigger picture. When you calculate the VIF for a feature, you are essentially running a regression of that feature against **all the other independent variables in your model**.

The <font color = 'cyan'>VIF score tells you how well that feature can be predicted by the others.</font> A high VIF indicates that the feature is redundant because its information is already contained within the other features. This makes VIF a more robust and reliable method for detecting multicollinearity in multiple regression models.

### <font color='gold'> Why would we use correlation instead of a VIF?</font> 

<font color = 'plum'>
Use **correlation** analysis instead of VIF when we want to understand the simple, pairwise relationship between two variables, especially for initial data exploration.

**Correlation**

Correlation is a measure of the linear relationship between **two variables**. It gives a single number (from -1 to 1) that tells the strength and direction of their relationship.

* **When to use it:** Use correlation for a quick, initial look at your data. It's great for identifying simple relationships and is a foundational step in data analysis.
* **Limitation:** Correlation is a **pairwise** measure. It cannot detect multicollinearity that involves a combination of three or more variables.



**VIF**

The **Variance Inflation Factor (VIF)** is a diagnostic tool used in **multiple regression** that measures how much a feature is explained by a linear combination of all the **other** features in the model.

* **When to use it:** Use VIF when you have a multiple regression model and you need to check for multicollinearity among all your features simultaneously. It's a more robust and comprehensive test.
* **Limitation:** VIF is not meant for initial data exploration or for understanding simple, two-variable relationships. It requires a full model context to be meaningful.

Use correlation for a **simple, two-variable check**, and use VIF for a **complex, multi-variable check** in a regression model.

--------

The **Variance Inflation Factor (VIF)** is typically used to detect **multicollinearity** among predictor variables in **linear regression models**, not classification models directly. However, here's a breakdown of when and how it might still be useful:

### ✅ When VIF *can* be useful in classification:
- **Logistic Regression**: Although it's a classification model, logistic regression is still a type of **generalized linear model (GLM)**. VIF can be used to assess multicollinearity among the independent variables in logistic regression.
- **Feature Selection**: If you're preparing features for any classification model (e.g., decision trees, random forests, SVMs), checking for multicollinearity using VIF can help reduce redundancy and improve model interpretability.

### ❌ When VIF is *less relevant*:
- **Tree-based models** (e.g., Random Forests, Gradient Boosting): These models are generally **not sensitive to multicollinearity**, so using VIF for feature selection may not be necessary.
- **Non-linear models**: For models like neural networks or k-NN, multicollinearity is less of a concern, and VIF may not provide meaningful insights.

### Alternatives to VIF:
If you're working with non-linear or tree-based classifiers, consider:
- **Feature importance scores** (e.g., from Random Forests or XGBoost)
- **Recursive Feature Elimination (RFE)**
- **Principal Component Analysis (PCA)** for dimensionality reduction